# Code-Snippets for PyTorch

## PyTest
Nice functionality to run test-cases on certain modules of your model.
Add the following content to a Python file and name it starting with 'test\_'. 
Every test function name will also begin with 'test\_'.

#### Parametrize
One can parametrize the test cases so they won't need to be 'copied' multiple times. Notice the 3 parameters provided in the decorator and their values as a list of tuples.

In [9]:
import pytest

def sum(a, b):
    return a + b

def pow(base, exponent):
    return base ** exponent

@pytest.mark.parametrize('a, b, expected',[
    (1,2,3),
    (4,5,9),
    (3,4,7)
])
def test_sum(a, b, expected):
    assert sum(a,b) == expected

#### Fixtures
Used for executing a step before the testing can commence. Notice the syntax of the decorator below. The function that is decorated with a fixture is something that will be executed whenever that function name is supplied as an argument to a test function. The return value substitutes for the argument of the test function.

In [ ]:
@pytest.fixture
def base():
    return 4

def test_pow(base):
    assert pow(base,1) == 4

#### Execution
Use the following command in the same directory as the above Python file. The flag '-v' indicates that a verbose output be shown.

In [ ]:
!pytest -v

## Progress Logging
Use one of the following functions to print the progress during training.
### Batch Progress Bar

In [16]:
def batch_progress_bar(batch_num, report_interval, last_loss):
    """Prints the progress until the next report."""
    progress = (((batch_num-1.0) % report_interval) + 1.0) / report_interval
    fill = int(progress * 40)
    print "\r\tBATCH [{}{}]: {} (Loss: {:.4f})".format(
        "=" * fill, 
        " " * (40 - fill), 
        batch_num, 
        last_loss)

batch_progress_bar(40,100,334.23)
batch_progress_bar(45,100,334.23)

	BATCH [================                        ]: 40 (Loss: 334.2300)
	BATCH [==================                      ]: 45 (Loss: 334.2300)


### Epoch Progress Bar

In [17]:
def epoch_progress_bar(epoch_num, total_epochs, train_loss, test_loss):
    """Prints the progress until the next report."""
    progress = (((epoch_num-1.0) % total_epochs) + 1.0) / total_epochs
    fill = int(progress * 40)
    print "\rEPOCH [{}{}]: {} (Avg. Training Loss: {:.4f}) (Test Loss: {:.4f})".format(
        "=" * fill, 
        " " * (40 - fill), 
        epoch_num, 
        train_loss,
        test_loss
    )

epoch_progress_bar(24,100,334.23,352.45)
epoch_progress_bar(30,100,334.23,352.45)

EPOCH [=========                               ]: 24 (Avg. Training Loss: 334.2300) (Test Loss: 352.4500)
EPOCH [============                            ]: 30 (Avg. Training Loss: 334.2300) (Test Loss: 352.4500)


### Saving

In [ ]:
def save_checkpoint_for_batch(checkpoint_path,
                              model,
                              name,
                              seed,
                              epoch_num,
                              batch_num,
                              losses,
                              costs,
                              seq_lengths):
    basename = "{}/{}-{}-epoch-{}-batch-{}".format(checkpoint_path, name, seed, epoch_num, batch_num)
    model_fname = basename + ".model"
    LOGGER.info("Saving model checkpoint to: '%s'", model_fname)
    torch.save(model.state_dict(), model_fname)

    # Save the training history for batch
    train_fname = basename + ".json"
    LOGGER.info("Saving model training history to '%s'", train_fname)
    content = {
        "loss": losses,
        "cost": costs,
        "seq_lengths": seq_lengths
    }
    open(train_fname, 'wt').write(json.dumps(content))